In [1]:
!ls

README.md		    miniconda.sh  quick_start_pytorch.ipynb   wandb
eliciting-latent-sentiment  miniconda3	  quick_start_pytorch_images


In [2]:
%cd eliciting-latent-sentiment

/notebooks/eliciting-latent-sentiment


In [3]:
!pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
!pip install transformer_lens
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git
# !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
# %pip install git+https://github.com/neelnanda-io/PySvelte.git
# %pip install typeguard==2.13.3

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-4rjfiwn2
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-4rjfiwn2
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 5afe6fed827592dd525490b81e213bc3e2241a4a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 104.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 117.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 34.3 MB/s eta 0:00

In [4]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [5]:
import einops
from functools import partial
import torch
from torch import Tensor
from torch.utils.data import DataLoader
from datasets import load_dataset
from jaxtyping import Float, Int, Bool
from typing import Dict, Iterable, List, Tuple, Union
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_dataset, tokenize_and_concatenate, get_act_name, test_prompt
from transformer_lens.hook_points import HookPoint
from tqdm.notebook import tqdm
import pandas as pd
from circuitsvis.activations import text_neuron_activations
from utils.store import load_array, save_html, save_array, is_file, get_model_name, clean_label, save_text

In [6]:
torch.set_grad_enabled(False)
device = "cuda"
MODEL_NAME = "EleutherAI/pythia-2.8b"
model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
    device=device,
)
model.name = MODEL_NAME

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-2.8b into HookedTransformer


In [87]:
BATCH_SIZE = 24
owt_data = load_dataset("stas/openwebtext-10k", split="train")
owt_dataset = tokenize_and_concatenate(owt_data, model.tokenizer)
owt_data_loader = DataLoader(
    owt_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True
)

Repo card metadata block was not found. Setting CardData to empty.


In [90]:
owt_dataset[0]

{'tokens': tensor([    0,    34, 11338,  ...,   773,   688,   247])}

In [75]:
from datasets import Dataset
import pandas as pd

# Define a function to read the text file and create a DataFrame
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Read lines and create a DataFrame
        lines = file.readlines()
        df = pd.DataFrame({'text': lines})
        return df

# Path to your text file
file_path = 'hp.txt'

# Read the text file into a DataFrame
text_df = read_text_file(file_path)

# Convert the DataFrame to a HuggingFace Dataset
text_dataset = Dataset.from_pandas(text_df)

# Concatenate all items in the 'text' column and 

In [125]:
from typing import List
from datasets import Dataset, Features, Sequence, Value
from transformers import AutoTokenizer
import torch

def tokenize_and_concatenate2(dataset, tokenizer, max_length=1024, column_name='text', add_bos_token=True):
    token_buffer = []
    final_batches = []
    
    for batch in dataset:
        text = batch[column_name]
        if add_bos_token:
            text = tokenizer.bos_token + text
        tokenized_text = tokenizer(text, add_special_tokens=False)['input_ids']
        eos_token_id = tokenizer.eos_token_id
        tokenized_text.append(eos_token_id)
        token_buffer.extend(tokenized_text)
        
        while len(token_buffer) >= max_length:
            final_batch = token_buffer[:max_length]
            token_buffer = token_buffer[max_length:]
            final_batches.append(final_batch)
    
    # Handle any remaining tokens
    if len(token_buffer) > 0:
        final_batches.append(token_buffer)
    
    # Convert list of batches to tensors
    final_batches = [torch.tensor(batch) for batch in final_batches]
    
    # Create a new dataset with specified features
    features = Features({"tokens": Sequence(Value("int64"))})
    final_dataset = Dataset.from_dict({"tokens": final_batches}, features=features)

    final_dataset.set_format(type="torch", columns=["tokens"])
    
    return final_dataset

# # Example usage
# tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Make sure the tokenizer has bos_token_id and eos_token_id
# text_dataset = Dataset.from_dict({"text": ["This is a sample text.", "Another sample text."]})  # Example dataset
# tokenized_dataset = tokenize_and_concatenate2(text_dataset, tokenizer, max_length=1024)



In [126]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer


# Define the batch size
BATCH_SIZE = 5

# Load a tokenizer (you'll need to specify the appropriate model)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b")
# set padding token
tokenizer.pad_token = tokenizer.eos_token

#dataset = text_dataset.map(lambda x: tokenize_and_concatenate(x, tokenizer))

dataset = tokenize_and_concatenate2(text_dataset, tokenizer, max_length=1024, column_name='text')

# Create a PyTorch DataLoader
train_data_loader = DataLoader(
    dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True
)

# Now, train_data_loader is ready to be used for training or analysis


In [127]:
dataset

Dataset({
    features: ['tokens'],
    num_rows: 6
})

In [128]:
dataset = dataset.select(range(5))

In [129]:
type(dataset[0]['tokens'])

torch.Tensor

In [22]:
def find_positions(tensor, token_ids=[11, 13]):
    positions = []
    for batch_item in tensor:
        token_positions = {token_id: [] for token_id in token_ids}
        for position, token in enumerate(batch_item):
            if token.item() in token_ids:
                token_positions[token.item()].append(position)
        positions.append([token_positions[token_id] for token_id in token_ids])
    return positions

In [23]:
def zero_attention_pos_hook(
    pattern: Float[Tensor, "batch head seq_Q seq_K"], hook: HookPoint,
    pos_by_batch: List[List[int]], layer: int = 0, head_idx: int = 0,
) -> Float[Tensor, "batch head seq_Q seq_K"]:
    """Zero-ablates an attention pattern tensor at a particular position"""
    assert 'pattern' in hook.name

    batch_size = pattern.shape[0]
    assert len(pos_by_batch) == batch_size

    for i in range(batch_size):
        for p in pos_by_batch[i]:
            pattern[i, head_idx, p, p] = 0
            
    return pattern



In [24]:
def names_filter(name: str):
    """Filter for the names of the activations we want to keep to study the resid stream."""
    return name.endswith('resid_post') or name == get_act_name('resid_pre', 0)

def get_layerwise_token_mean_activations(model: HookedTransformer, data_loader: DataLoader, token_id: int = 13) -> Float[Tensor, "layer d_model"]:
    """Get the mean value of a token across layers"""
    num_layers = model.cfg.n_layers
    d_model = model.cfg.d_model
    
    activation_sums = torch.stack([torch.zeros(d_model) for _ in range(num_layers)]).to(device)
    comma_counts = [0] * num_layers

    print(activation_sums.shape)

    token_mean_values = torch.zeros((num_layers, d_model))
    for _, batch_value in tqdm(enumerate(data_loader), total=100):
        
        batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=[13])

        _, cache = model.run_with_cache(
            batch_tokens, 
            names_filter=names_filter
        )

        
        for i in range(batch_tokens.shape[0]):
            for p in punct_pos[i][0]:
                for layer in range(num_layers):
                    activation_sums[layer] += cache[f"blocks.{layer}.hook_resid_post"][i, p, :]
                    comma_counts[layer] += 1

    for layer in range(num_layers):
        token_mean_values[layer] = activation_sums[layer] / comma_counts[layer]

    return token_mean_values

In [86]:
comma_mean_values = get_layerwise_token_mean_activations(model, data_loader, token_id=13)

NameError: name 'data_loader' is not defined

In [ ]:
comma_mean_values.shape

torch.Size([32, 2560])

In [15]:
save_array(comma_mean_values, 'comma_mean_values.npy', model)

'data/pythia-2.8b/comma_mean_values.npy'

In [25]:
# load the file
comma_mean_values = torch.from_numpy(load_array('comma_mean_values.npy', model)).to(device)

In [53]:
def plot_neuroscope(
    tokens: Int[Tensor, "batch pos"], centred: bool = False, activations: Float[Tensor, "pos layer 1"] = None,
    verbose=False,
):
    
    str_tokens = model.to_str_tokens(tokens, prepend_bos=False)

    if verbose:
        print(f"Tokens shape: {tokens.shape}")
  
    if centred:
        if verbose:
            print("Centering activations")
        layer_means = einops.reduce(activations, "pos layer 1 -> 1 layer 1", reduction="mean")
        layer_means = einops.repeat(layer_means, "1 layer 1 -> pos layer 1", pos=activations.shape[0])
        activations -= layer_means
    elif verbose:
        print("Activations already centered")
    assert (
        activations.ndim == 3
    ), f"activations must be of shape [tokens x layers x neurons], found {activations.shape}"
    assert len(str_tokens) == activations.shape[0], (
        f"tokens and activations must have the same length, found tokens={len(str_tokens)} and acts={activations.shape[0]}, "
        f"tokens={str_tokens}, "
        f"activations={activations.shape}"

    )
    return text_neuron_activations(
        tokens=str_tokens, 
        activations=activations,
        first_dimension_name="Layer (resid_pre)",
        second_dimension_name="Model",
        second_dimension_labels=["pythia-2.8b"],
    )

In [54]:


# def compute_modified_loss(model: HookedTransformer, data_loader: DataLoader) -> float:
#     total_loss = 0
#     loss_list = []
#     for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
#         batch_tokens = batch_value['tokens'].to(device)

#         # get positions of all 11 and 13 token ids in batch
#         punct_pos = find_positions(batch_tokens, token_ids=[13])

#         # get the loss for each token in the batch
#         initial_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)
        
#         # add hooks for the activations of the 11 and 13 tokens
#         for layer, head in heads_to_ablate:
#             ablate_punct = partial(zero_attention_pos_hook, pos_by_batch=punct_pos, layer=layer, head_idx=head)
#             model.blocks[layer].attn.hook_pattern.add_hook(ablate_punct)

#         # get the loss for each token when run with hooks
#         hooked_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)

#         # compute the percent difference between the two losses
#         loss_diff = (hooked_loss - initial_loss) / initial_loss

#         loss_list.append(loss_diff)

#     model.reset_hooks()
#     return loss_list, batch_tokens

In [55]:
from utils.ablation import ablate_resid_with_precalc_mean

heads_to_ablate = [(layer, head) for layer in range(model.cfg.n_layers) for head in range(model.cfg.n_heads)]

def compute_mean_ablation_modified_loss(model: HookedTransformer, data_loader: DataLoader, cached_means) -> float:
    total_loss = 0
    loss_list = []
    for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
        if isinstance(batch_value['tokens'], list):
            batch_tokens = torch.stack(batch_value['tokens']).to(device)
        else:
            batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=[13])

        # get the loss for each token in the batch
        initial_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)
        
        # add hooks for the activations of the 11 and 13 tokens
        for layer, head in heads_to_ablate:
            mean_ablate_comma = partial(ablate_resid_with_precalc_mean, cached_means=cached_means, pos_by_batch=punct_pos, layer=layer)
            model.blocks[layer].hook_resid_post.add_hook(mean_ablate_comma)

        # get the loss for each token when run with hooks
        hooked_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)

        # compute the percent difference between the two losses
        loss_diff = (hooked_loss - initial_loss) / initial_loss
        loss_list.append(loss_diff)

    model.reset_hooks()
    return loss_list

In [56]:
model.to_tokens(",")

tensor([[ 0, 13]], device='cuda:0')

In [130]:
# Create a subset of the dataset containing only the first ten items
subset_dataset = dataset.select(range(5))

# Create a new dataloader from the subset, converting the data to tensors
subset_data_loader = DataLoader(
    subset_dataset, batch_size=5, shuffle=False, drop_last=True
)
len(subset_data_loader)


1

In [131]:
loss_change_by_token = compute_mean_ablation_modified_loss(model, subset_data_loader, comma_mean_values)
#loss_change_by_token, batch_tokens = compute_modified_loss(model, data_loader)
#loss_change_by_token[11]

  0%|          | 0/1 [00:00<?, ?it/s]

In [132]:
for i in range(len(loss_change_by_token)):
    # add one column of zeros to the loss change tensor
    loss_change_by_token[i] = torch.cat([torch.zeros(loss_change_by_token[i].shape[0], 1).to(device), loss_change_by_token[i]], dim=1)

loss_change_by_token = torch.stack(loss_change_by_token).cpu()


In [134]:
loss_change_by_token.shape

torch.Size([1, 5, 1024])

In [135]:
loss_change_by_token_by_row = einops.rearrange(loss_change_by_token, "batch item token -> (batch item) token")
loss_change_by_token_by_row = loss_change_by_token_by_row.unsqueeze(2)
loss_change_by_token_by_row.shape

torch.Size([5, 1024, 1])

In [136]:
save_array(loss_change_by_token_by_row, 'loss_change_by_token_by_row_hp.npy', model)

'data/pythia-2.8b/loss_change_by_token_by_row_hp.npy'

In [137]:
loss_change_by_token_by_row.sum()

tensor(5147.2622)

In [138]:
dataset[0]

{'tokens': tensor([    0,    47, 18579,  ...,   428,   512,   689])}

In [143]:
from utils.neuroscope import plot_topk
loss_change_by_token = torch.from_numpy(load_array('loss_change_by_token_by_row_hp.npy', model))
plot_topk(activations=loss_change_by_token_by_row, dataloader=subset_data_loader, window_size=30, model=model, k=15, centred=False)

Top 15 most positive examples:
Example: 't, Activation: 197.4535, Batch: 0, Pos: 680
Example: 't, Activation: 194.9202, Batch: 4, Pos: 767
Example: ge, Activation: 181.4583, Batch: 3, Pos: 651
Example: 't, Activation: 148.7570, Batch: 4, Pos: 630
Example: 't, Activation: 137.7805, Batch: 1, Pos: 836
Example: 't, Activation: 137.1132, Batch: 2, Pos: 667
Example: 't, Activation: 133.3771, Batch: 4, Pos: 647
Example: 't, Activation: 115.0428, Batch: 1, Pos: 729
Example: 't, Activation: 114.2895, Batch: 3, Pos: 414
Example: 't, Activation: 113.6054, Batch: 4, Pos: 706
Example: 't, Activation: 102.0353, Batch: 4, Pos: 779
Example: 't, Activation: 98.2425, Batch: 4, Pos: 526
Example: icking, Activation: 95.8333, Batch: 3, Pos: 300
Example:  it, Activation: 75.2759, Batch: 3, Pos: 182
Example: ing, Activation: 68.4738, Batch: 4, Pos: 187


Top 15 most negative examples:
Example:  Dud, Activation: -0.7942, Batch: 3, Pos: 315
Example:  Dud, Activation: -0.7634, Batch: 1, Pos: 86
Example:  as, Activation: -0.7207, Batch: 2, Pos: 821
Example:  Dud, Activation: -0.6996, Batch: 0, Pos: 642
Example:  Dud, Activation: -0.6850, Batch: 1, Pos: 74
Example:  Harry, Activation: -0.6827, Batch: 4, Pos: 1002
Example:  Harry, Activation: -0.6675, Batch: 2, Pos: 584
Example:  Harry, Activation: -0.6560, Batch: 0, Pos: 809
Example:  hat, Activation: -0.6377, Batch: 4, Pos: 891
Example:  Dud, Activation: -0.6065, Batch: 2, Pos: 278
Example:  and, Activation: -0.5967, Batch: 4, Pos: 587
Example:  and, Activation: -0.5753, Batch: 2, Pos: 1004
Example: g, Activation: -0.5629, Batch: 1, Pos: 720
Example:  Harry, Activation: -0.5586, Batch: 0, Pos: 678
Example: leys, Activation: -0.5460, Batch: 4, Pos: 636


In [70]:
subset_data_loader.dataset[0]['tokens'].shape

AttributeError: 'list' object has no attribute 'shape'

In [35]:
plot_neuroscope(batch_tokens[11], activations=loss_change_by_token[11].unsqueeze(1).unsqueeze(2))

In [77]:
loss_change_by_token[11].max()

tensor(607.2508, device='cuda:0')

In [62]:
loss_change_by_token[11].unsqueeze(1).unsqueeze(2).shape

torch.Size([1024, 1, 1])

In [67]:
batch_tokens[11].unsqueeze(0)

tensor([[    0,   604, 40537,  ...,  4163,   846,  1534]], device='cuda:0')